# <center>MATHEMATICS AND STATISTICS FOR DATA SCIENCE</center>
<center>Thời gian: 180 phút</center>
<center>Ngày thi: 20/09/2020</center>
<center> ****** </center>
<center>Thí sinh: <b>TRẦN HOÀNG BÁCH  </b> </center>    
<center>   ---   </b> </center>  
<center>Câu 4: Kiểm định ANOVA </b> </center>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import scipy.stats as stats

from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
df = pd.read_csv('Data/One way ANOVA.txt', delimiter='\t')
df.head()

,A,B,C,D
0,93,85.0,100.0,96.0
1,120,45.0,75.0,58.0
2,65,80.0,65.0,95.0
3,105,28.0,40.0,90.0
4,115,75.0,73.0,65.0


In [3]:
# Check null-value
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       15 non-null     int64  
 1   B       10 non-null     float64
 2   C       12 non-null     float64
 3   D       9 non-null      float64
dtypes: float64(3), int64(1)
memory usage: 608.0 bytes


In [4]:
A = np.array(df['A'].dropna())
B = np.array(df['B'].dropna())
C = np.array(df['C'].dropna())
D = np.array(df['D'].dropna())

## Câu 4.1
Để kiểm định các quần thể có cùng phương sai, ta sử dụng Kiểm định Levene  

* H0: All Samples are from population with equal Variance
* Ha: Have at least 1 difference Variance

In [5]:
# Levene test
alpha = 0.05
t_lev, p_lev = stats.levene(A, B, C, D)
p_lev

0.5020818274234007

In [6]:
if p_lev > alpha:
    print('(p >  alpha) ==> FAIL TO REJECT the H0 that All Samples are from population with equal variances.')
else:
    print('(p <= alpha) ==> REJECT the H0 that All Samples are from population with equal variances.')

(p >  alpha) ==> FAIL TO REJECT the H0 that All Samples are from population with equal variances.


Nhận xét: Vì p > alpha nên Fail to Reject H0 "Các quần thể có cùng phương sai"  
==> Vậy nên với độ tin cậy 95%, có thể kết luận các quần thể có cùng phương sai

## Câu 4.2
### Để có thể thực hiện kiểm định về giả thuyết H0 "Các quần thể có cùng giá trị trung bình, ta thực hiện kiểm định One-way ANOVA  
Để thực hiện kiểm định One-way ANOVA, trước hết cần thoả 3 điều kiện sau:
* ĐK1: Các quan sát là độc lập

* ĐK2: Các quần thể có phân phối chuẩn

* ĐK3: Các quần thể có phương sai giống nhau

ĐK1: Vì không có thông tin gì về mối liên quan giữa A, B, C, D nên xem như các mẫu là độc lập. ĐK1 thoả mãn.  

ĐK2: Thoả mãn theo 4.1

ĐK2: Thoả mãn theo thông tin từ đề bài

==> Vậy có thể thực hiện kiểm định One-way ANOVA

In [7]:
# Cách 1: Tính toán truyền thống
k = 4
samples = []
samples.append(A.tolist())
samples.append(B.tolist())
samples.append(C.tolist())
samples.append(D.tolist())

## Số phần tử của mỗi nhóm
sizes = np.zeros(k)
for j in range(k): 
    sizes[j] = np.size(samples[j])

## Giá trị trung bình của mỗi mẫu
means = np.zeros(k)
for j in range(k):
    means[j] = np.mean(samples[j])

## Giá trị trung bình của tất cả các mẫu
meanT = np.mean(means)

## Các đại lượng BETWEEN groups: SSB, dfB
SSB = 0
for j in range(k):
    SSB += sizes[j] * np.power((means[j] - meanT), 2)
dfB = (k - 1)
print('SSB           : %.4f' %SSB)
print('dfB           : %d' %dfB)
   
## Các đại lượng WITHIN groups: SSW, dfW
SSW = 0
for j in range(k):
    SSWj = 0
    for i in range(int(sizes[j])):
        SSWj += np.power(samples[j][i] - means[j], 2)
    SSW += SSWj
dfW = int(np.sum(sizes) - k)
print('SSW           : %.4f' %SSW)
print('dfW           : %d' %dfW)

## Trị thống kê: F statistics
F = (SSB / dfB) / (SSW / dfW)
print('F statistic   : %.4f' %F)

SSB           : 12486.4848
dfB           : 3
SSW           : 11464.5722
dfW           : 42
F statistic   : 15.2479


In [8]:
critical_value = stats.f.ppf(q = 1 - alpha, dfn = dfB, dfd = dfW)
print('Critical value: %.4f' %critical_value)
conf_level     = stats.f.cdf(critical_value, dfn = dfB, dfd = dfW)

if (F < critical_value):
    print('(F <  critical value) ==> FAIL TO REJECT the H0 that the means are equal.')
else:
    print('(F >= critical value) ==> REJECT the H0 that the means are equal.')

Critical value: 2.8270
(F >= critical value) ==> REJECT the H0 that the means are equal.


In [9]:
# Cách 2: Tính toán truyền thống
f_ano, p_ano = stats.f_oneway(A, B, C, D)
f_ano

15.139053436390862

In [10]:
if p_ano > alpha:
    print('(p >  alpha) ==> FAIL TO REJECT the H0 that the means are equal.')
else:
    print('(p <= alpha) ==> REJECT the H0 that the means are equal.')

(p <= alpha) ==> REJECT the H0 that the means are equal.


Nhận xét: Cách 1 và 2 đều cho ra kết quả REJECT H0 "Các quần thể có cùng giá trị trung bình"  

==> Vậy nên với mức độ tin cậy 95%, có thể kết luận **Có tồn tại ít nhất 1 cặp quần thể có giá trị trung bình khác nhau**  

Để xét thêm từng cặp khác nhau ra sao, ta thực hiện hậu kiểm (post-hoc test), chọn **Tukey HSD** test

In [11]:
# Chuyển đổi dữ liệu để đưa vào form Tukey HSD test
df_melt = pd.melt(df.reset_index(), id_vars=['index'], value_vars=['A', 'B', 'C', 'D'])
df_melt = df_melt.dropna()
df_melt.head()

,index,variable,value
0,0,A,93.0
1,1,A,120.0
2,2,A,65.0
3,3,A,105.0
4,4,A,115.0


In [12]:
# Tukey HSD test
m_comp = pairwise_tukeyhsd(endog=df_melt['value'], groups=df_melt['variable'], alpha=alpha)
print(m_comp)

Multiple Comparison of Means - Tukey HSD, FWER=0.05  
group1 group2 meandiff p-adj   lower    upper   reject
------------------------------------------------------
     A      B -35.3667  0.001 -53.4103  -17.323   True
     A      C -36.9167  0.001 -54.0344 -19.7989   True
     A      D -11.7778 0.3415 -30.4132   6.8577  False
     B      C    -1.55    0.9 -20.4744  17.3744  False
     B      D  23.5889 0.0171   3.2814  43.8964   True
     C      D  25.1389 0.0068   5.6495  44.6283   True
------------------------------------------------------


#### Nhận xét: Ngoại trừ cặp (A,D) và (B,C), tất cả các cặp so sánh khác đều bác bỏ H0, chỉ ra các cặp xe đó có sự khác biệt đáng kể về mặt thống kê